<a href="https://colab.research.google.com/github/Iremguel/Fallstudie_Elektrifizierung_der_Logistik/blob/main/Stufe_2_(ohne_Ladeinfrastruktur).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie Elektrifizierung

In [ ]:
! pip install -q pyscipopt

In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

In [ ]:
! git clone https://github.com/Iremguel/Fallstudie_Elektrifizierung_der_Logistik.git

fatal: destination path 'Fallstudie_Elektrifizierung_der_Logistik' already exists and is not an empty directory.


In [ ]:
! ls

Fallstudie_Elektrifizierung_der_Logistik  sample_data


In [ ]:
! ls Daten

ls: cannot access 'Daten': No such file or directory


In [ ]:
folder = "Fallstudie_Elektrifizierung_der_Logistik/Daten/"

In [ ]:
chargers = pd.read_csv(f"{folder}chargers.csv", sep=";")

In [ ]:
diesel_trucks = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [ ]:
electric_trucks = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [ ]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

## Hilfunktionen

In [ ]:
def time_to_slot(t):
    """Wandelt HH:MM in 15-Minuten-Zeitschritte um"""
    h, m = map(int, t.split(":"))
    return (60 * h + m) // 15

routes["start"] = routes["starttime"].apply(time_to_slot)
routes["end"]   = routes["endtime"].apply(time_to_slot)

## Indexmengen

In [ ]:
R = list(routes["route_id"].unique()) # Routen

#LKWs
#K_D = list(diesel_trucks["truck_model"].unique()) # Diesel LKW-Typen
K_E = list(electric_trucks["truck_model"].unique()) # e-LKW Typen
#K = K_D + K_E # Gesamte LKW-Flotte

V = list(range(len(R)))      # physische LKWs (Ausprägung)

S = list(chargers["charger_model"].unique()) # Ladesäulentypen

T = list(range(96))  # 24h * 4

In [ ]:
DAYS = list(range(260))

## Parameter

In [ ]:
# Diesel-LKW-Parameter (nur ein Typ)
d = "ActrosL"

diesel_capex = diesel_trucks.set_index("truck_model").loc[truck, "capex_yearly"]
diesel_opex  = diesel_trucks.set_index("truck_model").loc[truck, "opex_yearly"]
diesel_kfz   = diesel_trucks.set_index("truck_model").loc[truck, "kfz_yearly"]
diesel_cons  = diesel_trucks.set_index("truck_model").loc[truck, "avg_diesel_per_100km"]

# Konstanten
diesel_price = 1.60  # €/Liter
toll_price   = 0.34  # €/km
DAYS = 260           # Betriebstage pro Jahr

# Fixkosten pro LKW (jährlich)
diesel_fixed_cost = diesel_capex + diesel_opex + diesel_kfz

In [ ]:
# Tourenparameter
route_start = routes.set_index("route_id")["start"].to_dict()
route_end   = routes.set_index("route_id")["end"].to_dict()
distance_total = routes.set_index("route_id")["distance_total"].to_dict()
distance_toll  = routes.set_index("route_id")["distance_toll"].to_dict()

In [ ]:
# e-LKWs
# Kosten und Erlöse
electric_capex = electric_trucks.set_index("truck_model")["capex_yearly"].to_dict()
electric_opex  = electric_trucks.set_index("truck_model")["opex_yearly"].to_dict()
thg_revenue    = electric_trucks.set_index("truck_model")["thg_yearly"].to_dict()

# Energie und Technik
electric_consumption = electric_trucks.set_index("truck_model")["avg_energy_kWh_per_100km"].to_dict()
truck_max_power      = electric_trucks.set_index("truck_model")["max_power"].to_dict()
soc_max              = electric_trucks.set_index("truck_model")["soc_max_kWh"].to_dict()
electricity_price = 0.25  # €/kWh

In [ ]:
# Routenkosten pro Jahr
diesel_route_cost = {}
for r in R:
    fuel = diesel_cons / 100 * distance_total[r] * diesel_price
    toll = distance_toll[r] * toll_price
    diesel_route_cost[r] = DAYS * (fuel + toll)

electric_route_cost = {}
for r in R:
    for k in K_E:
        energy_cost = (
            electric_consumption[k] / 100
            * distance_total[r]
            * electricity_price
        )
        electric_route_cost[r, k] = DAYS * energy_cost

## Modellinstanz ziehen

In [ ]:
model = Model()

## Entscheidungsvariablen

In [ ]:
# Tour–LKW-Zuordnung
# x[r,v] = 1 wenn Route r von LKW v gefahren wird
X_D = {}
for r in R:
    for v in V:
        X_D[r, v] = model.addVar(vtype="B", name=f"xD_{r}_{v}")

X_E = {}
for r in R:
    for v in V:
      for k in K_E:
        X_E[r, v, k] = model.addVar(vtype="B", name=f"xE_{r}_{v}_{k}")

In [ ]:
# y[v] = 1 wenn LKW v genutzt wird
Y_D = {}
for v in V:
    Y_D[v] = model.addVar(vtype="B", name=f"yD_{v}")

Y_E = {}
for v in V:
  for k in K_E:
    Y_E[v, k] = model.addVar(vtype="B", name=f"yE_{v}_{k}")

## Zielfunktion

In [ ]:
model.setObjective(
    # Diesel Fixkosten
    quicksum(diesel_fixed_cost * Y_D[v] for v in V)
    +
    # e-LKW Fixkosten minus THG
    quicksum(
        (electric_capex[k] + electric_opex[k] - thg_revenue[k]) * Y_E[v, k]
        for v in V for k in K_E
    )
    +
    # Variable Kosten
    quicksum(diesel_route_cost[r] * X_D[r, v] for r in R for v in V)
    +
    quicksum
        (electric_route_cost[r, k] * X_E[r, v, k]
        for r in R for v in V for k in K_E
    )
    ,
    sense="minimize"

)

## Nebenbedingungen

In [ ]:
# Jede Route wird genau einmal gefahren (Diesel ODER e-LKW)
for r in R:
    model.addCons(
        quicksum(X_D[r, v] for v in V)
        + quicksum(X_E[r, v, k] for v in V for k in K_E)
        == 1,
        name=f"route_coverage_{r}"
    )


In [ ]:
# Route nur wenn LKW existiert
for r in R:
    for v in V:
        model.addCons(X_D[r, v] <= Y_D[v])
        for k in K_E:
            model.addCons(X_E[r, v, k] <= Y_E[v, k])

In [ ]:
# Ein physischer LKW ist ENTWEDER Diesel ODER ein e-LKW-Typ
for v in V:
    model.addCons(
        Y_D[v] + quicksum(Y_E[v, k] for k in K_E) <= 1,
        name=f"one_vehicle_type_{v}"
    )

In [ ]:
# Keine zeitliche Überlappung auf demselben LKW
for v in V:
    for i, r1 in enumerate(R):
        for r2 in R[i+1:]:
            if not (
                route_end[r1] <= route_start[r2]
                or route_end[r2] <= route_start[r1]
            ):
                model.addCons(
                    X_D[r1, v] + X_D[r2, v]
                    + quicksum(X_E[r1, v, k] + X_E[r2, v, k] for k in K_E)
                    <= 1,
                    name=f"overlap_{r1}_{r2}_{v}"
                )

## Lösung

In [ ]:
#model.redirectOutput()

In [ ]:
model.optimize()

In [ ]:
print("\n==============================")
print("OPTIMALE LÖSUNG (OHNE LADEN)")
print("==============================")
print(f"Gesamtkosten pro Jahr: {model.getObjVal():,.2f} €")

print("\nEingesetzte Fahrzeuge:")
for v in V:
    if model.getVal(Y_D[v]) > 0.5:
        print(f"LKW {v}: Diesel")
    for k in K_E:
        if model.getVal(Y_E[v, k]) > 0.5:
            print(f"LKW {v}: e-LKW {k}")

print("\nRoutenplanung:")
for r in R:
    for v in V:
        if model.getVal(X_D[r, v]) > 0.5:
            print(f"Route {r} → Diesel LKW {v}")
        for k in K_E:
            if model.getVal(X_E[r, v, k]) > 0.5:
                print(f"Route {r} → e-LKW {k} (LKW {v})")


OPTIMALE LÖSUNG (OHNE LADEN)
Gesamtkosten pro Jahr: 851,418.35 €

Eingesetzte Fahrzeuge:
LKW 3: Diesel
LKW 5: Diesel
LKW 8: Diesel
LKW 9: e-LKW eActros400
LKW 10: e-LKW eActros400
LKW 11: Diesel
LKW 12: e-LKW eActros400
LKW 13: e-LKW eActros400
LKW 14: Diesel
LKW 15: e-LKW eActros400
LKW 16: Diesel
LKW 17: Diesel
LKW 18: e-LKW eActros400
LKW 19: Diesel

Routenplanung:
Route t-4 → e-LKW eActros400 (LKW 10)
Route t-5 → e-LKW eActros400 (LKW 15)
Route t-6 → e-LKW eActros400 (LKW 18)
Route s-1 → e-LKW eActros400 (LKW 9)
Route s-2 → Diesel LKW 19
Route s-3 → e-LKW eActros400 (LKW 13)
Route s-4 → e-LKW eActros400 (LKW 12)
Route w1 → Diesel LKW 11
Route w2 → Diesel LKW 14
Route w3 → Diesel LKW 8
Route w4 → Diesel LKW 5
Route w5 → Diesel LKW 16
Route w6 → Diesel LKW 3
Route w7 → Diesel LKW 17
Route r1 → e-LKW eActros400 (LKW 18)
Route r2 → e-LKW eActros400 (LKW 13)
Route r3 → e-LKW eActros400 (LKW 12)
Route h3 → e-LKW eActros400 (LKW 15)
Route h4 → e-LKW eActros400 (LKW 10)
Route k1 → e-LKW e